# Agents as Tools with Strands Agents


"Agents as Tools" is an architectural pattern in AI systems where specialized AI agents are wrapped as callable functions (tools) that can be used by other agents. This creates a hierarchical structure where:

1. A primary "orchestrator" agent handles user interaction and determines which specialized agent to call

2. Specialized "tool agents" perform domain-specific tasks when called by the orchestrator

This approach mimics human team dynamics, where a manager coordinates specialists, each bringing unique expertise to solve complex problems. Rather than a single agent trying to handle everything, tasks are delegated to the most appropriate specialized agent.



## Key Benefits and Core Principles

The "Agents as Tools" pattern offers several advantages:

- Separation of concerns: Each agent has a focused area of responsibility, making the system easier to understand and maintain
- Hierarchical delegation: The orchestrator decides which specialist to invoke, creating a clear chain of command
- Modular architecture: Specialists can be added, removed, or modified independently without affecting the entire system
- Improved performance: Each agent can have tailored system prompts and tools optimized for its specific task


In [ ]:
!pip install -r requirements.txt

In [ ]:
import os

from strands import Agent, tool
from strands_tools import file_write

In this module we will be creating an orchestrator based multi-agent workflow. 

<div style="text-align:left">
    <img src="images/architecture.png" width="75%" />
</div>

We will also explore `use_llm` which allows use to create nested agents.

## Research Agent

Lets first create a basic reasearch assistant with http_request tool. 

In [ ]:
RESEARCH_ASSISTANT_PROMPT = """You are a specialized research assistant. Focus only on providing
factual, well-sourced information in response to research questions.
Always cite your sources when possible."""

In [ ]:
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
    # tools=[http_request]  # Here you can enable an agentic ai search tool
)

query = "Overview of Amazon Bedrock and its features"
# Call the agent and return its response
response = research_agent(query)

Now we can wrap this agent as a tool. Allowing other agents to interact with it. 

####  Best Practices for Agent as Tools

When implementing the "Agents as Tools" pattern with Strandly AI:

1. Clear tool documentation: Write descriptive docstrings that explain the agent's expertise
2. Focused system prompts: Keep each specialized agent tightly focused on its domain
3. Proper response handling: Use consistent patterns to extract and format responses
4. Tool selection guidance: Give the orchestrator clear criteria for when to use each specialized agen

In [ ]:
@tool
def research_assistant(query: str) -> str:
    """
    Process and respond to research-related queries.

    Args:
        query: A research question requiring factual information

    Returns:
        A detailed research answer with citations
    """
    try:
        # Strands agents makes it easy to create a specialized agent
        research_agent = Agent(
            system_prompt=RESEARCH_ASSISTANT_PROMPT,
        )

        # Call the agent and return its response
        response = research_agent(query)
        return str(response)
    except Exception as e:
        return f"Error in research assistant: {str(e)}"

Now lets follow the best practices and create `product_recommendation_assistant`, `trip_planning_assistant`, and `orchestrator` agent.

### Product Recommendation Assistant

In [ ]:
@tool
def product_recommendation_assistant(query: str) -> str:
    """
    Handle product recommendation queries by suggesting appropriate products.

    Args:
        query: A product inquiry with user preferences

    Returns:
        Personalized product recommendations with reasoning
    """
    try:
        product_agent = Agent(
            system_prompt="""You are a specialized product recommendation assistant.
            Provide personalized product suggestions based on user preferences. Always cite your sources.""",
        )
        # Call the agent and return its response
        response = product_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in product recommendation: {str(e)}"

In [ ]:
product_recommendation_assistant("Product recommendations for flying cars")

### Trip Planning Assistant

In [ ]:
@tool
def trip_planning_assistant(query: str) -> str:
    """
    Create travel itineraries and provide travel advice.

    Args:
        query: A travel planning request with destination and preferences

    Returns:
        A detailed travel itinerary or travel advice
    """
    try:
        travel_agent = Agent(
            system_prompt="""You are a specialized travel planning assistant.
            Create detailed travel itineraries based on user preferences.""",
        )
        # Call the agent and return its response
        response = travel_agent(query)

        return str(response)
    except Exception as e:
        return f"Error in trip planning: {str(e)}"

### Orchestrator Agent

In [ ]:
# Define orchestrator system prompt with clear tool selection guidance
MAIN_SYSTEM_PROMPT = """
You are an assistant that routes queries to specialized agents:
- For research questions and factual information → Use the research_assistant tool
- For product recommendations and shopping advice → Use the product_recommendation_assistant tool
- For travel planning and itineraries → Use the trip_planning_assistant tool
- For simple questions not requiring specialized knowledge → Answer directly

Always select the most appropriate tool based on the user's query.
"""

In [ ]:
# Strands Agents allows easy integration of agent tools
orchestrator = Agent(
    system_prompt=MAIN_SYSTEM_PROMPT,
    tools=[
        research_assistant,
        product_recommendation_assistant,
        trip_planning_assistant,
        file_write,
    ],
)

In [ ]:
# Example: E-commerce Customer Service System
customer_query = (
    "I'm looking for hiking boots. Write the final response to current directory."
)

os.environ["BYPASS_TOOL_CONSENT"] = "true"

# The orchestrator automatically determines this requires multiple specialized agents
response = orchestrator(customer_query)

Lets look at the messages of the orchestrator. Here you can see the agent decided to use the sub-agent as tool

In [ ]:
orchestrator.messages

In [ ]:
customer_query = "Can you help me plan my trip to Patagonia"

response = orchestrator(customer_query)

In [ ]:
orchestrator.messages

### Calling multiple agents 

In [ ]:
orchestrator.messages = []

In [ ]:
query = "Can you do a research on spain? Also help me plan a 7 day trip."

orchestrator(query)

Behind the scenes, the orchestrator will:
1. First call the `research_assistant`
2. Then call `trip_planning_assistant`
3. Combine these specialized responses into a cohesive answer that addresses both the queries

### Sequential Agent Communication Pattern


The agent tool can also combine multiple agents together. In this example we will provide output of `research_agent` to `summary_agent` and return the summarized response.

In [ ]:
 # define the user query
topic = "generative Ai"
# Create a research agent
research_agent = Agent(
    system_prompt=RESEARCH_ASSISTANT_PROMPT,
)
# Create a summarization agent
summary_agent = Agent(
    system_prompt="""
    You are a summarization specialist focused on distilling complex information into clear, concise summaries.
    Your primary goal is to take detailed information and extract the key points, main arguments, and critical data.
    You should maintain the accuracy of the original content while making it more digestible.
    Focus on clarity, brevity, and highlighting the most important aspects of the information.
    """,
)

print("Multiple agents created successfully!")
print(f"\n🔍 RESEARCH AGENT working on: {topic}\n") 
try:
    # Agent 1: Invoke research agent
    research_response = research_agent(
        f"Please gather comprehensive information about {topic}."
    )
    research_text = research_response.message.content[0]["text"]
    print("\n✂️ SUMMARY AGENT distilling the research\n")
    
    # Agent 2: Ask the summary agent to create a concise summary
    summary_response = summary_agent(
        f"Please create a concise summary of this research: {research_text}"
    )
    summary_text = summary_response.message.content[0]["text"]
    
    print(summary_text)
except Exception as e:
    print(f"Error in research assistant: {str(e)}")

## Congrats!

You've learned how to use agents as tools in Strands Agents to create more complex agentic applications